# envar, mounts, services 

In [3]:
from gql import  gql, Client
from gql.transport.requests import RequestsHTTPTransport

dash_enterprise_host = "dash-playground.plotly.host"
username = "developers"
username_api_key = "faBhA8WwjuLpC8QoEulU"

target_app = "aa-tngo-oil-and-gas"
target_app_config


transport = RequestsHTTPTransport(
    url =  "https://{dash_enterprise_host}/Manager/graphql".format(dash_enterprise_host = dash_enterprise_host), 
    auth = (username, username_api_key), 
    use_json = True
)

client = Client(transport = transport)

# Querying target app...
print(f"Querying...{target_app}")

api_results = []
apps = []

query_string = ("""
{{
    apps(name: "{target_app}", allApps:true) {{
        apps {{
            linkedServices {{
                serviceType
            }}
            mounts {{
                targetDir
                hostDir
            }}
            environmentVariables {{
                name
                value
            }}
        }}
    }}
}}""".format(target_app = target_app))

api_call = client.execute(gql(query_string))
apps = api_call["apps"]["apps"]
api_results.extend(apps)

# Parsing queries by api_results, and appending them to their respective lists, then zipping lists for formatting
print("Parsing...", end='\n')

## Parsing linkServices
linkserv_results = api_results[0]["linkedServices"]
linkserv_name = []
linkserv_type = []

if len(linkserv_results) != 0:
    print("Parsing linkServices...", end='\n')
    for i in range(len(linkserv_results)):
        linkserv_name.append(linkserv_results[i]["name"])
        linkserv_type.append(linkserv_results[i]["serviceType"])
else:
    print("Parsing linkServices...SKIPPED!", end='\n')

linkserv_zip = dict(zip(linkserv_type, linkserv_name))

for k,v in linkserv_zip.items():
    print("linkServices: ", k, ": ", v)

## Parsing mounts
mounts_results = api_results[0]["mounts"]
mounts_targetdir = []
mounts_hostdir = []

if len(mounts_results) != 0:
    print("Parsing mounts...")
    for i in range(len(mounts_results)):
        mounts_targetdir.append(mounts_results[i]["targetDir"])
        mounts_hostdir.append(mounts_results[i]["hostDir"])
else:
    print("Parsing mounts...SKIPPED!")

mounts_zip = dict(zip(mounts_targetdir, mounts_hostdir))

for k,v in mounts_zip.items():
    print("mounts: ", k, ": ", v)

## Parsing environmentVariables
envar_results = api_results[0]["environmentVariables"]
envar_name = []
envar_value = []

if len(envar_results) != 0:
    print("Parsing environmentVariables...", end='\n')
    for i in range(len(envar_results)):
        envar_name.append(envar_results[i]["name"])
        envar_value.append(envar_results[i]["value"])
else:
    print("Parsing environmentVariables...SKIPPED!", end='\n')

envar_zip = dict(zip(envar_name, envar_value))
for k,v in envar_zip.items():
    print("environmentVariable: ", k, ": ", v)

# Filtering/Formatting/Calling API
if len(linkserv_results) != 0:
    print("Formatting Linked Services...", end='\n')
    filtered_dict = dict()
    for key,value in linkserv_zip.items():
        query_string = """
            addService(serviceType: {service_type}, name: "{app_name}-{service_type}") {{
            service {{
                name
                serviceType
                created
            }}
            error
        }}  
        """.format(service_type=key, app_name=app_name)
        # client.execute(gql(query_string))
        print("Adding linkedServices: {service_type}, {app_name}-{service_type}".format(service_type=key, app_name=app_name))
else:
    print("Formatting Linked Services...SKIPPED!", end='\n')


Querying...aa-tngo-oil-and-gas
Parsing...
Parsing linkServices...SKIPPED!
Parsing mounts...SKIPPED!
Parsing environmentVariables...
environmentVariable:  DASH_APP_NAME :  aa-tngo-oil-and-gas
environmentVariable:  DASH_DOMAIN_BASE :  dash-playground.plotly.host
environmentVariable:  DASH_LOGOUT_URL :  /Manager/api/logout
environmentVariable:  DASH_PATH_ROUTING :  1
environmentVariable:  DASH_SECRET_KEY :  XbWK8whnVmF9j7UIwue0YfGOkrAeshCO
environmentVariable:  DASH_STREAMBED_DIRECT_IP :  172.17.0.1
environmentVariable:  DOKKU_APP_RESTORE :  1
environmentVariable:  DOKKU_APP_TYPE :  herokuish
environmentVariable:  DOKKU_PROXY_PORT :  60940
environmentVariable:  DOKKU_PROXY_PORT_MAP :  http:60940:5000  
environmentVariable:  GIT_REV :  7546f51c864a37e7e752c6ad528682ca6c1e2b0b
environmentVariable:  NO_VHOST :  1
environmentVariable:  SALESFORCE_CONSUMER_SECRET :  B58F10210F5ABCB11884CB21738EE73CAFB8FC34A72A41DD387EBBA6FBF6D271
environmentVariable:  SCRIPT_NAME :  /aa-tngo-oil-and-gas
Format